In [1]:
import pandas as pd
# Load our sample CSV data as DataFrame
df = pd.read_csv('wilder-rsi-data.csv', header=0).set_index(['period'])


In [2]:
# View the result
print(df.head())


        price
period       
1       54.80
2       56.80
3       57.85
4       59.85
5       60.57


In [3]:
# View summary
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 1 to 38
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   38 non-null     float64
dtypes: float64(1)
memory usage: 608.0 bytes
None


In [4]:
# Calculate Price Differences
df['diff'] = df.diff(1)
# View Results
print(df)

        price  diff
period             
1       54.80   NaN
2       56.80  2.00
3       57.85  1.05
4       59.85  2.00
5       60.57  0.72
6       61.10  0.53
7       62.17  1.07
8       60.60 -1.57
9       62.35  1.75
10      62.15 -0.20
11      62.35  0.20
12      61.45 -0.90
13      62.80  1.35
14      61.37 -1.43
15      62.50  1.13
16      62.57  0.07
17      60.80 -1.77
18      59.37 -1.43
19      60.35  0.98
20      62.35  2.00
21      62.17 -0.18
22      62.55  0.38
23      64.55  2.00
24      64.37 -0.18
25      65.30  0.93
26      64.42 -0.88
27      62.90 -1.52
28      61.60 -1.30
29      62.05  0.45
30      60.05 -2.00
31      59.70 -0.35
32      60.90  1.20
33      60.25 -0.65
34      58.27 -1.98
35      58.70  0.43
36      57.72 -0.98
37      58.10  0.38
38      58.20  0.10


In [5]:
# Calculate Avg. Gains/Losses
df['gain'] = df['diff'].clip(lower=0).round(2)
df['loss'] = df['diff'].clip(upper=0).abs().round(2)
# View Result
print(df)

        price  diff  gain  loss
period                         
1       54.80   NaN   NaN   NaN
2       56.80  2.00  2.00  0.00
3       57.85  1.05  1.05  0.00
4       59.85  2.00  2.00  0.00
5       60.57  0.72  0.72  0.00
6       61.10  0.53  0.53  0.00
7       62.17  1.07  1.07  0.00
8       60.60 -1.57  0.00  1.57
9       62.35  1.75  1.75  0.00
10      62.15 -0.20  0.00  0.20
11      62.35  0.20  0.20  0.00
12      61.45 -0.90  0.00  0.90
13      62.80  1.35  1.35  0.00
14      61.37 -1.43  0.00  1.43
15      62.50  1.13  1.13  0.00
16      62.57  0.07  0.07  0.00
17      60.80 -1.77  0.00  1.77
18      59.37 -1.43  0.00  1.43
19      60.35  0.98  0.98  0.00
20      62.35  2.00  2.00  0.00
21      62.17 -0.18  0.00  0.18
22      62.55  0.38  0.38  0.00
23      64.55  2.00  2.00  0.00
24      64.37 -0.18  0.00  0.18
25      65.30  0.93  0.93  0.00
26      64.42 -0.88  0.00  0.88
27      62.90 -1.52  0.00  1.52
28      61.60 -1.30  0.00  1.30
29      62.05  0.45  0.45  0.00
30      

In [6]:
# Get initial Averages
window_length = 14

df['avg_gain'] = df['gain'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]
df['avg_loss'] = df['loss'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]

In [7]:
# Get WMS averages
# Average Gains
for i, row in enumerate(df['avg_gain'].iloc[window_length+1:]):
    df['avg_gain'].iloc[i + window_length + 1] =\
        (df['avg_gain'].iloc[i + window_length] *
         (window_length - 1) +
         df['gain'].iloc[i + window_length + 1])\
        / window_length
# Average Losses
for i, row in enumerate(df['avg_loss'].iloc[window_length+1:]):
    df['avg_loss'].iloc[i + window_length + 1] =\
        (df['avg_loss'].iloc[i + window_length] *
         (window_length - 1) +
         df['loss'].iloc[i + window_length + 1])\
        / window_length
# View initial results
print(df[window_length-1:window_length+5])

        price  diff  gain  loss  avg_gain  avg_loss
period                                             
14      61.37 -1.43  0.00  1.43       NaN       NaN
15      62.50  1.13  1.13  0.00  0.842857  0.292857
16      62.57  0.07  0.07  0.00  0.787653  0.271939
17      60.80 -1.77  0.00  1.77  0.731392  0.378943
18      59.37 -1.43  0.00  1.43  0.679150  0.454019
19      60.35  0.98  0.98  0.00  0.700639  0.421589


In [8]:
# Calculate RS Values
df['rs'] = df['avg_gain'] / df['avg_loss']

In [9]:
# Calculate RSI
df['rsi'] = 100 - (100 / (1.0 + df['rs']))
# View Result
print(df)

        price  diff  gain  loss  avg_gain  avg_loss        rs        rsi
period                                                                  
1       54.80   NaN   NaN   NaN       NaN       NaN       NaN        NaN
2       56.80  2.00  2.00  0.00       NaN       NaN       NaN        NaN
3       57.85  1.05  1.05  0.00       NaN       NaN       NaN        NaN
4       59.85  2.00  2.00  0.00       NaN       NaN       NaN        NaN
5       60.57  0.72  0.72  0.00       NaN       NaN       NaN        NaN
6       61.10  0.53  0.53  0.00       NaN       NaN       NaN        NaN
7       62.17  1.07  1.07  0.00       NaN       NaN       NaN        NaN
8       60.60 -1.57  0.00  1.57       NaN       NaN       NaN        NaN
9       62.35  1.75  1.75  0.00       NaN       NaN       NaN        NaN
10      62.15 -0.20  0.00  0.20       NaN       NaN       NaN        NaN
11      62.35  0.20  0.20  0.00       NaN       NaN       NaN        NaN
12      61.45 -0.90  0.00  0.90       NaN       NaN

In [10]:
# Save dataframe as CSV
df.to_csv('wilder-rsi-pandas-output.csv')

In [13]:
import pandas_ta as ta
# Load the data
df = pd.read_csv('wilder-rsi-data.csv', header=0).set_index(['period'])

In [18]:
# Calculate the RSI via pandas_ta
df.ta.rsi(close='price', length=14, append=True)

# View the result
print(df)

        price  RSI_38     RSI_14
period                          
1       54.80     NaN        NaN
2       56.80     NaN        NaN
3       57.85     NaN        NaN
4       59.85     NaN        NaN
5       60.57     NaN        NaN
6       61.10     NaN        NaN
7       62.17     NaN        NaN
8       60.60     NaN        NaN
9       62.35     NaN        NaN
10      62.15     NaN        NaN
11      62.35     NaN        NaN
12      61.45     NaN        NaN
13      62.80     NaN        NaN
14      61.37     NaN        NaN
15      62.50     NaN  68.701706
16      62.57     NaN  68.936534
17      60.80     NaN  57.241560
18      59.37     NaN  49.879205
19      60.35     NaN  54.224444
20      62.35     NaN  61.550568
21      62.17     NaN  60.610379
22      62.55     NaN  61.932388
23      64.55     NaN  68.016644
24      64.37     NaN  66.979072
25      65.30     NaN  69.562559
26      64.42     NaN  64.426106
27      62.90     NaN  56.645732
28      61.60     NaN  50.975691
29      62

In [15]:
df.ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

In [19]:
length = 14

In [22]:
# pandas_ta gain/loss average calculations
positive_avg = ta.rma(positive, length=length)
negative_avg = ta.rma(negative, length=length)
# the underlying ram function where
# 'close' is the column from which values
# are calculated ('gain' and 'loss') using
# pandas ewn() method.
ta.rma = close.ewm(alpha=alpha, min_periods=length).mean()

NameError: name 'positive' is not defined